# Sea Ice Comparisons to Observations

This script shows how to load and plot sea ice concentration from CICE output .....

Requirements: The conda/analysis3 module from /g/data/hh5/public/modules. 

**Access OM-2 Runs**

In [ ]:
RUNS={
    '01deg':['01deg_jra55v140_iaf_cycle4', '01deg_jra55v140_iaf_cycle4_jra55v150_extension'],
    '025deg':['025deg_jra55_iaf_omip2_cycle6'],
    '1deg':['1deg_jra55_iaf_omip2_cycle6']
}

In [ ]:
VARS=['hi_m', 'hs_m', 'aice_m']#ice thickness, snow thickness, ice area fraction
VARS_2D=['area_t','geolat_t','geolon_t']

**OBS Data:**

In [129]:
ST='1979'
ET='2022'
OBS_TIME_SLICE=slice(ST,ET)
sh_obs_url='https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4shmday'
nh_obs_url='https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4nhmday'
_obs_data_dir='/g/data/gv90/P6_data/'
#Using NSIDC, see below for dets

Climatology Time Slice:


In [1]:
CLIMAT_SLICE=slice('1981,2010')
TIME_SLICE=slice('1979','2023')

In [81]:
_notebook_path= !echo /g/data/$PROJECT/$USER/

_notebook_path=_notebook_path[0]+'/cosima-recipes'

%run '{_notebook_path}/DocumentedExamples/CICE_Loaders.ipynb'

Check conda/analysis version

In [1]:
! conda env list

# conda environments:
#
                         /g/data/gv90/as2285/miniconda3
                         /g/data/gv90/as2285/miniconda3/envs/sea-ice-extremes
                         /g/data/gv90/as2285/miniconda3/envs/sea-ice-plots
                         /g/data/gv90/as2285/miniconda3/envs/sea-ice-tracker
base                  *  /g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07



Create a work directory:

Load modules:

In [ ]:
import intake

import matplotlib.pyplot as plt
from dask.distributed import Client
from datetime import timedelta
import cf_xarray as cfxr
import cartopy.crs as ccrs
import xarray as xr
import xesmf
import numpy as np

from datatree import DataTree,map_over_subtree


In [6]:
client = Client()

In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 8
Total threads: 48,Total memory: 188.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34023,Workers: 8
Dashboard: /proxy/8787/status,Total threads: 48
Started: Just now,Total memory: 188.56 GiB
Comm: tcp://127.0.0.1:42067,Total threads: 6
Dashboard: /proxy/45373/status,Memory: 23.57 GiB
Nanny: tcp://127.0.0.1:45989,


Open the catalog

In [17]:
catalog = intake.cat.access_nri

In [18]:
catalog

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{ocean, seaIce}","{3mon, 3hr, 1mon, fx, 1day}","{dyu, alidr_ai_m, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, aice_m, uhrho_et, lprec, ULON, pme_net, vert_pv, total_ocean_river_heat, age_global, fsalt_ai_m, surface_salt, total_o..."
01deg_jra55v140_iaf,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{ocean, seaIce}","{1day, 1mon, fx}","{dyu, alidr_ai_m, aicen, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, aice_m, lprec, ULON, pme_net, vert_pv, total_ocean_river_heat, age_global, fsalt_ai_m, surface_salt, salt_surfa..."
01deg_jra55v140_iaf_cycle2,{ACCESS-OM2},{Cycle 2 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{ocean, seaIce}","{1day, 1mon, fx}","{dyu, alidr_ai_m, aicen, fsurf_ai, fcondtop_ai, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, aice_m, lprec, ULON, pme_net, vert_pv, total_ocean_river_heat, age_global, fsalt_ai_m, s..."
01deg_jra55v140_iaf_cycle3,{ACCESS-OM2},{Cycle 3 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{ocean, seaIce}","{1day, 1mon, fx}","{dyu, alidr_ai_m, fsurf_ai, fcondtop_ai, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, aice_m, lprec, ULON, pme_net, vert_pv, total_ocean_river_heat, age_global, fsalt_ai_m, surface_..."
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{ocean, seaIce}","{6hr, 3hr, 1mon, fx, 1day}","{caco3_xflux_adv, lprec, ULON, wdet100, src10, no3, uvel_h, total_ocean_pme_river, caco3_yflux_adv, frzmlt_m, total_ocean_salt, skl_Nit, o2_yflux_adv, dardg2dt_m, total_ocean_runoff_heat, dyt, alv..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},"{ocean, seaIce}","{1day, 1mon, subhr, fx}","{caco3_xflux_adv, dyu, alidr_ai_m, fsurf_ai, fcondtop_ai, fe, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, aice_m, surface_dic, lprec, zoo, ULON, wdet100, src10, vert_pv, pme_net, t..."
01deg_jra55v150_iaf_cycle1,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0 OMIP2 interannual forcing},"{ocean, seaIce}","{1day, 1mon, fx}","{dyu, mld, sfc_salt_flux_ice, aice_m, lprec, ULON, pme_net, age_global, surface_salt, blkmask, pme_river, sea_level_sq, surface_pot_temp, melt, fprec, geolat_c, dyt, geolat_t, diff_cbt_t, sens_hea..."
025deg_jra55_iaf_omip2_cycle1,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, 1yr, fx}","{dyu, alidr_ai_m, patm_t, fsurf_ai, diff_cbt_s, fcondtop_ai, salt_vdiffuse_k33, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, neutral_gm_temp, aice_m, strtltx_m, lprec, strtlty_m, UL..."
025deg_jra55_iaf_omip2_cycle2,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, 1yr, fx}","{dyu, alidr_ai_m, patm_t, fsurf_ai, diff_cbt_s, fcondtop_ai, salt_vdiffuse_k33, HTN, mld, fmeltt_ai_m, total_ocean_melt, sfc_salt_flux_ice, neutral_gm_temp, aice_m, strtltx_m, lprec, strtlty_m, UL..."


In [ ]:
%%time

si_dt = DataTree.from_dict(dict([
    (iRun, open_by_run(iRun)) for iRun in RUNS.keys()
]))

Subset to the date range we have obs for

In [ ]:
si_dt=si_dt.sel(time=TIME_SLICE)

In [ ]:
si_dt['01deg']

In [ ]:
@map_over_subtree
def calc_area(ds):
    sic=ds.aice_m
    in_range=sic.where(
        (sic>=0.15)*(sic<=1)
    )

    return (in_range*ds.area_t).cf.sum(['X','Y']).to_dataset(name='si_area').load()

In [ ]:
si_area_dt=calc_area(
    si_dt.sel(yt_ocean=slice(-90,0))
)

In [ ]:
si_area_dt

In [ ]:
@map_over_subtree
def min_and_max(ds):
    def min_and_max_year(da):
        result = xr.Dataset()
        result['min']=da.min()
        result['max']=da.max()
        return result
    annual_min_max_ds=ds.si_area.groupby('time.year').apply(min_and_max_year)
    return annual_min_max_ds
    
sia_min_max=min_and_max(si_area_dt)

In [ ]:
for iGroup in sia_min_max.groups[1:]:
    ds=sia_min_max[iGroup].ds
    ds['min'].plot(label=iGroup[1:])
    ds['max'].plot(label=iGroup[1:])

plt.legend()